In [ ]:
import atdata
import atdata._local_dev as al

local_index = al.Index()
x: al.IndexEntry = local_index.list()[0]

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [4]:
x.sample_schema.json_schema

{'$id': 'https://schema.dev/fake-schema.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'fake-schema',
 'description': 'Blue Blah',
 'type': 'object',
 'properties': {'property_a': {'default': 5, 'type': 'integer'},
  'property_b': {'type': 'string'}}}

In [6]:
from pydantic import create_model, Field

In [7]:
json_schema = x.sample_schema.json_schema

# Map JSON Schema types and constraints to Pydantic fields
fields = {}
for prop_name, prop_details in json_schema["properties"].items():
    field_type = None
    if prop_details["type"] == "string":
        if prop_details.get("format") == "email":
            field_type = str  # Pydantic's EmailStr can be used for stronger validation
        else:
            field_type = str
    elif prop_details["type"] == "integer":
        field_type = int
    # Add more type mappings as needed

    field_args = {}
    if "maxLength" in prop_details:
        field_args["max_length"] = prop_details["maxLength"]
    if "minimum" in prop_details:
        field_args["ge"] = prop_details["minimum"]
    # Add more constraint mappings (e.g., pattern, minLength, maximum, etc.)

    if prop_name in json_schema.get("required", []):
        fields[prop_name] = (field_type, Field(**field_args))
    else:
        fields[prop_name] = (field_type, Field(None, **field_args)) # Optional field

In [27]:
_FakeSchema = create_model( 'FakeSchema', **fields )
class FakeSchema( _FakeSchema, atdata.PackableSample ):
    pass

In [37]:
x = FakeSchema( property_a = 2, property_b = 'hello' )

In [38]:
x_prime = FakeSchema.from_bytes( x.as_wds['msgpack'] )

In [39]:
x_prime

FakeSchema(property_a=2, property_b='hello')